# A* grid

In [ ]:
import heapq 

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0
        self.h = 0
        self.f = 0

    def __lt__(self, other):
        return self.f < other.f
    
def heuristic(a, b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])
    
def astar_search(grid, start, end):

    open = []
    closed = set()

    start_node = Node(start)
    end_node = Node(end)

    heapq.heappush(open, start_node)

    while open:

        current_node = heapq.heappop(open)
        closed.add(current_node.position)

        if current_node.position == end_node.position:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]
        
        (x, y) = current_node.position

        neighbours = [(x+1, y), (x-1, y), (x, y-1), (x, y+1)]

        for next_pos in neighbours:
            (nx, ny) = next_pos
            if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] == 0:
                if next_pos in closed:
                    continue

                neighbor = Node(next_pos, current_node)
                neighbor.g = current_node.g + 1
                neighbor.h = heuristic(neighbor.position, end_node.position)
                neighbor.f = neighbor.g + neighbor.h

                if all(open_node.position != neighbor.position or open_node.f > neighbor.f for open_node in open):
                    heapq.heappush(open, neighbor)
    return None 

    
grid = [
    [0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

start = (0, 0)
end = (4, 4)

path = astar_search(grid, start, end)

print("Path: ", path)

# A* 8-puzzle


In [ ]:
import heapq


goal_state = [
    [1, 2, 3],
    [8, 0, 4],
    [7, 6, 5]
]

def state_to_tuple(state):
    res = []
    for row in state:
        for num in row:
            res.append(num)
    return tuple(res)

def misplaced_tiles(state):
    cnt = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0 and state[i][j] != goal_state[i][j]:
                cnt += 1
    return cnt

def get_neighbors(state):

    neighbors = []

    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                x = i
                y = j
                break 

    tmp_neighbors = [(x+1, y), (x-1, y), (x, y-1), (x, y+1)]

    for next_pos in tmp_neighbors:
        nx, ny = next_pos
        if 0 <= nx < 3 and 0 <= ny < 3:
            neighbor_state = [row[:] for row in state]
            neighbor_state[nx][ny], neighbor_state[x][y] = state[x][y], state[nx][ny]
            neighbors.append(neighbor_state)
    return neighbors


def print_state(state):
    for row in state:
        print(row)
    print()

def a_star(start_state):

    open = []
    closed = set()

    count = 0
    g_score = {state_to_tuple(start_state): 0}

    heapq.heappush(open, (0, count, start_state))
    parent = {}

    step = 0

    while open:
        f, _, current = heapq.heappop(open)
        current_tuple = state_to_tuple(current)
        closed.add(current_tuple)

        g = g_score[current_tuple]
        h = misplaced_tiles(current)

        print(f"step: {step}: Expanding node with g(n)={g}, h(n)={h}, f(n)={f}")
        print_state(current)
        step += 1

        if current == goal_state:
            print("Reached goal state\n")
            path = [current]
            while current_tuple in parent:
                current = parent[current_tuple]
                current_tuple = state_to_tuple(current)
                path.append(current)
            return path[::-1]
        
        for neighbor in get_neighbors(current):
            neighbor_tuple = state_to_tuple(neighbor)
            g_n = g+1

            if neighbor_tuple in closed:
                continue
            if g_n < g_score.get(neighbor_tuple, float('inf')):
                g_score[neighbor_tuple] = g_n
                h_n = misplaced_tiles(neighbor)
                f_n = g_n + h_n
                count += 1
                heapq.heappush(open, (f_n, count, neighbor))
                parent[neighbor_tuple] = current 

                print(f"--Neighbor with g={g_n}, h={h_n}, f={f_n}:")
                print_state(neighbor)
    print("No soln found")
    return None 


start = [
    [2, 8, 3],
    [1, 6, 4],
    [7, 0, 5]
]

soln = a_star(start)

if soln:
    print("Final soln found in : ", len(soln)-1, "moves:\n")

    for i, step in enumerate(soln):
        print(f"step {i}: ")
        print_state(step)
else:
    print("No soln found!")

# A* graph

In [ ]:
from queue import PriorityQueue

# Graph definition: node -> {neighbor: cost}
graph = {
    'S': {'A': 1, 'G': 10},
    'A': {'B': 2, 'C': 1},
    'C': {'G': 4, 'D': 3},
    'D': {'G': 2},
    'B': {'D': 5},
    'G': {}
}

# Heuristic values: estimated cost from node to goal
heuristic = {
    'S': 5,
    'A': 3,
    'B': 4,
    'C': 2,
    'D': 6,
    'G': 0
}

def a_star_search(start, goal):
    pq = PriorityQueue()
    pq.put((heuristic[start], 0, start, [start]))  # (f, g, current_node, path_so_far)

    open_set = {start}
    closed_set = set()

    step = 0

    while not pq.empty():
        f, g, current, path = pq.get()
        open_set.discard(current)
        closed_set.add(current)

        print(f"\nStep {step}: Visiting node '{current}'")
        print("OPEN:", list(open_set))
        print("CLOSED:", list(closed_set))

        if current == goal:
            print("\nGoal reached!")
            print("Final Path:", " -> ".join(path))
            print("Total Cost (f):", f)
            return path, f

        for neighbor, cost in graph[current].items():
            if neighbor not in closed_set:
                g_new = g + cost
                f_new = g_new + heuristic[neighbor]
                pq.put((f_new, g_new, neighbor, path + [neighbor]))
                open_set.add(neighbor)

        step += 1

    print("Goal not reachable.")
    return None, float('inf')


# Run the algorithm
a_star_search('S', 'G')


# BFS 8-puzzle

In [ ]:
import heapq

goal_state = [
    [1, 2, 3],
    [8, 0, 4],
    [7, 6, 5]
]

def state_to_tuple(state):
    return tuple(num for row in state for num in row)

def misplaced_tiles(state):
    return sum(
        1 for i in range(3) for j in range(3)
        if state[i][j] != 0 and state[i][j] != goal_state[i][j]
    )

def get_neighbors(state):
    neighbors = []
    x, y = [(i, row.index(0)) for i, row in enumerate(state) if 0 in row][0]
    directions = [(-1,0), (1,0), (0,-1), (0,1)]
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < 3 and 0 <= ny < 3:
            new_state = [row[:] for row in state]
            new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
            neighbors.append(new_state)
    return neighbors

def print_state(state):
    for row in state:
        print(" ".join(str(num) for num in row))
    print()

def best_first_search(start_state):
    open_list = []
    heapq.heappush(open_list, (misplaced_tiles(start_state), start_state))
    closed = set()
    parent = {}
    closed.add(state_to_tuple(start_state))

    steps = 0
    while open_list:
        h, current = heapq.heappop(open_list)

        print(f"Step {steps}: Expanding node with h(n) = {h}")
        print_state(current)

        if current == goal_state:
            print("Goal state reached!\n")
            path = [current]
            while state_to_tuple(current) in parent:
                current = parent[state_to_tuple(current)]
                path.append(current)
            return path[::-1]

        neighbors = get_neighbors(current)
        for neighbor in neighbors:
            neighbor_tuple = state_to_tuple(neighbor)
            if neighbor_tuple not in closed:
                closed.add(neighbor_tuple)
                parent[neighbor_tuple] = current
                h_n = misplaced_tiles(neighbor)
                heapq.heappush(open_list, (h_n, neighbor))

                print(f"  Neighbor with h={h_n}:")
                print_state(neighbor)

        steps += 1

    print("No solution found.")
    return None

# Example initial state
start_state = [
    [2, 8, 3],
    [1, 6, 4],
    [7, 0, 5]
]

solution = best_first_search(start_state)

if solution:
    print("Solution path (may not be optimal):\n")
    for i, step in enumerate(solution):
        print(f"Move {i}:")
        print_state(step)
else:
    print("No path to goal found.")


# BFS - Graph

In [ ]:
import heapq

# Sample graph represented as an adjacency list
graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['G'],
    'F': [],
    'G': []
}

# Heuristic function (e.g., estimated distance to goal)
heuristic = {
    'A': 6,
    'B': 4,
    'C': 5,
    'D': 999,  # High value: dead end
    'E': 2,
    'F': 4,
    'G': 0     # Goal node has heuristic 0
}

# Best-First Search function
def best_first_search(graph, start, goal):
    visited = set()
    pq = []
    heapq.heappush(pq, (heuristic[start], start))
    parent = {start: None}

    while pq:
        h, current = heapq.heappop(pq)
        print(f"Visiting node: {current}, h(n) = {h}")

        if current == goal:
            print("\n Goal found!")
            # Reconstruct path
            path = []
            while current:
                path.append(current)
                current = parent[current]
            return path[::-1]

        visited.add(current)
        for neighbor in graph[current]:
            if neighbor not in visited:
                parent[neighbor] = current
                heapq.heappush(pq, (heuristic[neighbor], neighbor))

    print("No path found.")
    return None

# Run the search
start_node = 'A'
goal_node = 'G'
path = best_first_search(graph, start_node, goal_node)

if path:
    print("\n Path to goal:", " → ".join(path))
